In [77]:
import sqlite3
import pandasql as ps

import pandas as pd
import numpy as np

import openpyxl

In [100]:
#read in source tables 
DB_PATH = r'D:FIA\FIADB_CA.db'

con = sqlite3.connect(DB_PATH)
# grab NC stands
stands = pd.read_sql_query("select STAND_CN, STAND_ID, INV_YEAR, REGION, FOREST from FVS_STANDINIT_COND where VARIANT = 'NC'", con)

In [101]:
# scrape stand_id so remeasurement ids match
#stands['INV_YEAR'] = stands['INV_YEAR'].astype(int)
stands['STAND_ID'] = stands['STAND_ID'].str[0:3] + stands['STAND_ID'].str[10:]

In [107]:
# drop stands with no remeasurment data
stands = stands.sort_values(by=['STAND_ID', 'INV_YEAR', 'REGION', 'FOREST'])
remeasured_stands = stands[stands.duplicated(subset=['STAND_ID'], keep=False)]

In [109]:
# grab the trees and cond tables
cond = pd.read_sql_query("select CN, PLT_CN, CONDID from COND", con)
trees = pd.read_sql_query("select PLT_CN, CONDID, INVYR, TREE, STATUSCD, SPCD, DIA, HT, VOLCFNET, VOLCFGRS from TREE", con)

In [110]:
# use cond table to get plt_cn to match with trees table
trees_id = trees.merge(cond, how='left', on=['PLT_CN', 'CONDID'])

In [114]:
# grab tress for stands in our remeasured stands database
stand_trees = ps.sqldf("""select PLT_CN, CONDID, TREE, STATUSCD, SPCD, DIA, HT, VOLCFNET, VOLCFGRS, STAND_ID, REGION, FOREST from trees_id t join remeasured_stands s on t.CN = s.STAND_CN AND t.INVYR = s.INV_YEAR""")

In [112]:
# # now we need the fia equation for region and species
# species = stand_trees['SPCD'].unique()
# species.sort()
# species_db = pd.DataFrame(species, columns=['SPCD'])

# # grab the species reference table from FIA
# ref_species = pd.read_sql_query("select SPCD, COMMON_NAME, SPECIES_SYMBOL from REF_SPECIES", con)

# # filter to relevant species
# spec_ref = species_db.merge(ref_species, how='left', on=['SPCD'])

# ## create matrix for fia equation

In [115]:
# export to excel for use in National Volume Estimator Library (NVEL)
stand_trees.to_excel(r'D:\fvs_calibration\data\interim\NC_trees_for_NVEL.xlsx')